In [1]:
from importlib import reload
#reload(Utilities)
#reload(clm)
# NOTE: To reload a class imported as, e.g., 
# from module import class
# One must call:
#   1. import module
#   2. reload module
#   3. from module import class

import sys, os
import re
import string
import copy

import pandas as pd
import numpy as np
from pandas.api.types import is_numeric_dtype, is_datetime64_dtype, is_timedelta64_dtype
from scipy import stats
import datetime
import time
from natsort import natsorted, ns, natsort_keygen
from packaging import version

import itertools
import copy
import pyodbc
#---------------------------------------------------------------------
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
import matplotlib.ticker as ticker
from matplotlib import dates
#---------------------------------------------------------------------
sys.path.insert(0, os.path.realpath('..'))
import Utilities_config
#-----
import CommonLearningMethods as clm
#-----
from MeterPremise import MeterPremise
#-----
from AMI_SQL import AMI_SQL, DfToSqlMap
from AMINonVee_SQL import AMINonVee_SQL
from AMIEndEvents_SQL import AMIEndEvents_SQL
from AMIUsgInst_SQL import AMIUsgInst_SQL
from DOVSOutages_SQL import DOVSOutages_SQL
#-----
from GenAn import GenAn
from AMINonVee import AMINonVee
from AMIEndEvents import AMIEndEvents
from AMIUsgInst import AMIUsgInst
from DOVSOutages import DOVSOutages
from OutageDAQ import OutageDAQ, OutageDAQPrBL
#---------------------------------------------------------------------
sys.path.insert(0, Utilities_config.get_sql_aids_dir())
import Utilities_sql
import TableInfos
from TableInfos import TableInfo
from SQLElement import SQLElement
from SQLElementsCollection import SQLElementsCollection
from SQLSelect import SQLSelectElement, SQLSelect
from SQLFrom import SQLFrom
from SQLWhere import SQLWhereElement, SQLWhere
from SQLJoin import SQLJoin, SQLJoinCollection
from SQLGroupBy import SQLGroupByElement, SQLGroupBy
from SQLHaving import SQLHaving
from SQLOrderBy import SQLOrderByElement, SQLOrderBy
from SQLQuery import SQLQuery
from SQLQueryGeneric import SQLQueryGeneric
#---------------------------------------------------------------------
#sys.path.insert(0, os.path.join(os.path.realpath('..'), 'Utilities'))
sys.path.insert(0, Utilities_config.get_utilities_dir())
import Utilities
import Utilities_df
from Utilities_df import DFConstructType
import Utilities_dt
import Plot_Box_sns
import GrubbsTest
from CustomJSON import CustomEncoder, CustomWriter

# -----------------------------------------------------------------------------------------------
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# -----------------------------------------------------------------------------------------------

In [5]:
#----------------------------------------------------------------------------------------------------
# VARIABLES TO BE SET BY USER!
#----------------------------------------------------------------------------------------------------
save_dfs_to_file   = True
read_dfs_from_file = False
save_end_events    = True

#-------------------------
# run_date is used to collect all results from a given acquisiton run together.
# As such, run_date should be set to the first date of the acquisition run, and
#   SHOULD NOT be changed for each individual date in a run (which typically lasts
#   over the course of days/weeks)
run_date = '20240621' # Date of data acquisition

#-------------------------
# date_0 = '2020-01-01'
# date_1 = '2020-12-31'

# date_0 = '2022-01-01'
# date_1 = '2022-01-07'
# window_width = datetime.timedelta(days=1)

date_0 = '2023-04-01' # Lower limit for end events
date_1 = '2024-03-31' # Upper limit for end events
window_width = datetime.timedelta(days=31)

#-------------------------
run_using_slim = True

#--------------------------------------------------
# NOTE: below, states and opcos should be consistent!
#       i.e., e.g., if states='OH', then opcos should be 'oh' (or None, I suppose)
#-------------------------
# states used to 
#   (1) find transformers which suffered at least one outage from DOVS
#   (2) find all transformers from MeterPremise
# states can be:
#   - a single string, e.g. 'OH'
#   - a list of strings, e.g., ['OH', 'WV']
#   - None
# NOTE: states tend to be upper-case!
states=['OH']

#-------------------------
# opcos used with AMIEndEvents to
#  (1) find the premise numbers which recorded an event between date_0 and date_1.
#  (2) selection/acquisiton of end_device_events
# opcos can be:
#   - a single string, e.g. 'oh'
#   - a list of strings, e.g., ['oh', 'tx']
#   - None
# NOTE: opcos tend to be lower-case!
# NOTE: Acceptable opcos appear to be: ['ap', 'im', 'oh', 'pso', 'swp', 'tx']
opcos='oh'

#-------------------------
# cities = None
cities = ['COLUMBUS']

#--------------------------------------------------
trsf_pole_nbs_to_ignore = [' ', 'TRANSMISSION', 'PRIMARY', 'NETWORK']

# TODO!!!!!!!!!!!!!!!!!!
single_zip_xfmrs_only = True

In [ ]:
# # UNCOMMENT AFTER DONE CHECKING EVS_SUM
# ############### NEW
# outg_daq = OutageDAQPrBL(
#     run_date                = run_date, 
#     date_0                  = date_0, 
#     date_1                  = date_1, 
#     collect_evs_sum_vw      = False,  
#     save_sub_dir            = 'NoOutgs_Pristine', 
#     window_width            = window_width, 
#     states                  = states, 
#     opcos                   = opcos, 
#     cities                  = cities, 
#     single_zip_xfmrs_only   = single_zip_xfmrs_only, 
#     trsf_pole_nbs_to_ignore = trsf_pole_nbs_to_ignore, 
#     save_end_events         = save_end_events, 
#     save_dfs_to_file        = save_dfs_to_file, 
#     read_dfs_from_file      = read_dfs_from_file, 
#     base_dir                = os.path.join(
#         Utilities.get_local_data_dir(), 
#         r'dovs_and_end_events_data'
#     ), 
#     run_using_slim          = run_using_slim
# )

In [ ]:
# # UNCOMMENT AFTER DONE CHECKING EVS_SUM
# outg_daq.build_or_load_df_outage_location_ids(verbose=True)
# outg_daq.build_or_load_df_xfmrs_all_outg(verbose=True)
# outg_daq.build_or_load_trsf_pole_zips_info(verbose=True)
# outg_daq.build_or_load_df_mp_no_outg(verbose=True)
# outg_daq.build_or_load_pns_with_end_events(verbose=True)
# outg_daq.build_or_load_df_mp_no_outg_w_events_slim(verbose=True)

In [ ]:
# # UNCOMMENT AFTER DONE CHECKING EVS_SUM
# outg_daq.collect_events(
#     batch_size=None, 
#     verbose=True, 
#     n_update=1
# )

# CHECKING EVS_SUM METHOD VS ORIGINAL

In [ ]:
############### NEW
outg_daq = OutageDAQPrBL(
    run_date                = run_date, 
    date_0                  = date_0, 
    date_1                  = date_1, 
    collect_evs_sum_vw      = False,  
    save_sub_dir            = 'PristineBaseline', 
    window_width            = window_width, 
    states                  = states, 
    opcos                   = opcos, 
    cities                  = cities, 
    single_zip_xfmrs_only   = single_zip_xfmrs_only, 
    trsf_pole_nbs_to_ignore = trsf_pole_nbs_to_ignore, 
    save_end_events         = save_end_events, 
    save_dfs_to_file        = save_dfs_to_file, 
    read_dfs_from_file      = read_dfs_from_file, 
    base_dir                = os.path.join(
        Utilities.get_local_data_dir(), 
        r'dovs_and_end_events_data'
    ), 
    run_using_slim          = run_using_slim
)

In [ ]:
outg_daq.build_or_load_df_outage_location_ids(verbose=True)
outg_daq.build_or_load_df_xfmrs_all_outg(verbose=True)
outg_daq.build_or_load_trsf_pole_zips_info(verbose=True)
outg_daq.build_or_load_df_mp_no_outg(verbose=True)
outg_daq.build_or_load_pns_with_end_events(verbose=True)
outg_daq.build_or_load_df_mp_no_outg_w_events_slim(verbose=True)

In [ ]:
# Chop down DF size for quick run comp
if run_using_slim:
    outg_daq.df_mp_no_outg_w_events_slim = outg_daq.df_mp_no_outg_w_events_slim.iloc[:1000]
else:
    outg_daq.df_mp_no_outg_w_events = outg_daq.df_mp_no_outg_w_events.sort_values(by=['no_outg_rec_nb', 'trsf_pole_nb', 'prem_nb', 'start_date'], ignore_index=True).iloc[:10000]

In [ ]:
outg_daq.collect_events(
    batch_size=None, 
    verbose=True, 
    n_update=1
)

In [ ]:
############### NEWEST
outg_daq = OutageDAQPrBL(
    run_date                = run_date, 
    date_0                  = date_0, 
    date_1                  = date_1, 
    collect_evs_sum_vw      = True,  
    save_sub_dir            = 'NoOutgs_Pristine', 
    window_width            = window_width, 
    states                  = states, 
    opcos                   = opcos, 
    cities                  = cities, 
    single_zip_xfmrs_only   = single_zip_xfmrs_only, 
    trsf_pole_nbs_to_ignore = trsf_pole_nbs_to_ignore, 
    save_end_events         = save_end_events, 
    save_dfs_to_file        = save_dfs_to_file, 
    read_dfs_from_file      = read_dfs_from_file, 
    base_dir                = os.path.join(
        Utilities.get_local_data_dir(), 
        r'dovs_and_end_events_data'
    ), 
    run_using_slim          = run_using_slim
)

In [ ]:
outg_daq.build_or_load_df_outage_location_ids(verbose=True)
outg_daq.build_or_load_df_xfmrs_all_outg(verbose=True)
outg_daq.build_or_load_trsf_pole_zips_info(verbose=True)
outg_daq.build_or_load_df_mp_no_outg(verbose=True)
outg_daq.build_or_load_pns_with_end_events(verbose=True)
outg_daq.build_or_load_df_mp_no_outg_w_events_slim(verbose=True)

In [ ]:
# Chop down DF size for quick run comp
if run_using_slim:
    outg_daq.df_mp_no_outg_w_events_slim = outg_daq.df_mp_no_outg_w_events_slim.iloc[:1000]
else:
    outg_daq.df_mp_no_outg_w_events = outg_daq.df_mp_no_outg_w_events.sort_values(by=['no_outg_rec_nb', 'trsf_pole_nb', 'prem_nb', 'start_date'], ignore_index=True).iloc[:10000]

In [ ]:
outg_daq.collect_events(
    batch_size=None, 
    verbose=True, 
    n_update=1
)

In [ ]:
assert(0)

# END CHECKING EVS_SUM METHOD VS ORIGINAL

# OLD DEV

### OutagesDAQ constructor

In [9]:
cities

['COLUMBUS']

In [11]:
#----------------------------------------------------------------------------------------------------
#----------------------------------------------------------------------------------------------------
# DFs will be saved in save_dir_base
# Collection of end events files will be saved in os.path.join(save_dir_base, 'EndEvents')
save_dir_base = os.path.join(
    Utilities.get_local_data_dir(), 
    r'dovs_and_end_events_data', 
    run_date, 
    f"{date_0.replace('-','')}_{date_1.replace('-','')}", 
    'PristineBaseline_OLD'
)
#-------------------------
end_events_save_args = dict(
    save_to_file=save_end_events, 
    save_dir = os.path.join(save_dir_base, 'EndEvents'), 
    save_name=r'end_events.csv', 
    index=True
)
#-------------------------
print(f"save_dir_base = {save_dir_base}")
print('end_events_save_args')
for k,v in end_events_save_args.items():
    print(f"\t{k} : {v}")
#-------------------------
if save_dfs_to_file or save_end_events:
    if not os.path.exists(save_dir_base):
        os.makedirs(save_dir_base)
    #-----
    if save_end_events and not os.path.exists(end_events_save_args['save_dir']):
        os.makedirs(end_events_save_args['save_dir'])

save_dir_base = C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\20240621\20230401_20240331\PristineBaseline_OLD
end_events_save_args
	save_to_file : True
	save_dir : C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\20240621\20230401_20240331\PristineBaseline_OLD\EndEvents
	save_name : end_events.csv
	index : True


In [13]:
#----------------------------------------------------------------------------------------------------
#----------------------------------------------------------------------------------------------------
assert(save_dfs_to_file+read_dfs_from_file <=1) # Should never both read and write!
assert(pd.to_datetime(date_1)-pd.to_datetime(date_0) > window_width)
#--------------------------------------------------
if not read_dfs_from_file:
    conn_outages = Utilities.get_utldb01p_oracle_connection()
    conn_aws = Utilities.get_athena_prod_aws_connection()

# -----------------------------------------------------------------------------------------------
# -----------------------------------------------------------------------------------------------

# ----------------------------------------------------------------------------------------------------
# A. Find transformers which didn't experienced an outage between date_0 and date_1
# ----------------------------------------------------------------------------------------------------

### build_or_load_df_outage_location_ids

In [17]:
if read_dfs_from_file:
    df_outage_location_ids = pd.read_pickle(os.path.join(save_dir_base, 'df_outage_location_ids.pkl'))
else:
    #--------------------------------------------------
    # First, find all transformers which HAVE experienced an outage
    start=time.time()
    sql_outage_location_ids = DOVSOutages_SQL.build_sql_find_outage_xfmrs(
        mjr_mnr_cause=None, 
        include_premise=False, 
        date_range=[date_0, date_1], 
        states=states, 
        opcos=opcos, 
        cities=cities
    )
    sql_stmnt_outage_location_ids = sql_outage_location_ids.get_sql_statement()
    #-------------------------
    print('-----'*20+'\nFinding all transformers which HAVE experienced an outage\n'+'-----'*10)
    print(f'sql_stmnt_outage_location_ids:\n{sql_stmnt_outage_location_ids}\n\n')
    #-----
    df_outage_location_ids = pd.read_sql_query(sql_stmnt_outage_location_ids, conn_outages)
    #-----
    print(f'time = {time.time()-start}\n'+'-----'*20)
    #-------------------------
    if save_dfs_to_file:
        df_outage_location_ids.to_pickle(os.path.join(save_dir_base, 'df_outage_location_ids.pkl'))


DOVSOutages_SQL: Found opco in kwargs
Converting to OPERATING_UNIT_ID

Input opcos = ['oh']
OPERATING_UNIT_IDs used = [3]
----------------------------------------------------------------------------------------------------
Finding all transformers which HAVE experienced an outage
--------------------------------------------------
sql_stmnt_outage_location_ids:
SELECT
	DISTINCT(DOV.LOCATION_ID)
FROM DOVSADM.DOVS_OUTAGE_FACT DOV
	LEFT OUTER JOIN DOVSADM.DOVS_MASTER_GEO_DIM DOV1 ON DOV.OPERATING_UNIT_ID=DOV1.OPRTG_UNT_ID AND DOV.STATE_ABBR_TX=DOV1.STATE_ID AND DOV.OPCO_NBR=DOV1.OPCO_ID AND DOV.DISTRICT_NB=DOV1.DISTRICT_ID AND DOV.SRVC_CNTR_NB=DOV1.AREA_ID AND DOV.GIS_CRCT_NB=DOV1.GIS_CIRCUIT_ID
	LEFT OUTER JOIN DOVSADM.DOVS_OUTAGE_ATTRIBUTES_DIM DOV2 ON DOV.OUTG_REC_NB=DOV2.OUTG_REC_NB
	LEFT OUTER JOIN DOVSADM.DOVS_CLEARING_DEVICE_DIM DOV3 ON DOV.DEVICE_CD=DOV3.DEVICE_CD
	LEFT OUTER JOIN DOVSADM.DOVS_EQUIPMENT_TYPES_DIM DOV4 ON DOV.EQUIPMENT_CD=DOV4.EQUIPMENT_CD
	LEFT OUTER JOIN DOVSADM.

C:\Temp\ipykernel_31748\2503891294.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_outage_location_ids = pd.read_sql_query(sql_stmnt_outage_location_ids, conn_outages)


time = 7.103327751159668
----------------------------------------------------------------------------------------------------


### build_or_load_df_xfmrs_all_outg

In [22]:
if read_dfs_from_file:
    df_xfmrs_all     = pd.read_pickle(os.path.join(save_dir_base, 'df_xfmrs_all.pkl'))
    df_xfmrs_outg    = pd.read_pickle(os.path.join(save_dir_base, 'df_xfmrs_outg.pkl'))
    df_xfmrs_no_outg = pd.read_pickle(os.path.join(save_dir_base, 'df_xfmrs_no_outg.pkl'))
else:
    #----------------------------------------------------------------------------------------------------
    # Next, get the set of all transformers.
    # Then, we will be able to find the set which did not suffer an outage 
    #   as set(df_xfmrs_all).difference(set(df_outage_location_ids))
    #----------------------------------------------------------------------------------------------------
    print('-----'*20+'\nFinding set of all transformers\n'+'-----'*10)
    start=time.time()
    #-----
    sql_xfmrs_all = MeterPremise.build_sql_meter_premise(
        cols_of_interest = ['DISTINCT(trsf_pole_nb)'], 
        states           = states, 
        opcos            = opcos, 
        cities           = cities, 
        from_table_alias = None
    )
    # If OPCOs is None/not supplied, then SQLQuery object will be returned (original, default behavior)
    # If OPCOs is supplied, then string will be returned.
    # In either case, we need sql_xfmrs_all to be a str before inputting to pd.read_sql
    assert(Utilities.is_object_one_of_types(sql_xfmrs_all, [SQLQuery, str]))
    if isinstance(sql_xfmrs_all, SQLQuery):
        sql_xfmrs_all = sql_xfmrs_all.get_sql_statement()
    #-----
    df_xfmrs_all = pd.read_sql(sql_xfmrs_all, conn_aws) 
    #-----
    print(f'sql_xfmrs_all:\n{sql_xfmrs_all}\n\n')
    
    #--------------------------------------------------
    # Get no outage collection as those in df_xfmrs_all without trsf_pole_nb matching those in df_outage_location_ids
    #   i.e., set(df_xfmrs_all).difference(set(df_outage_location_ids))
    print(f"Determining no outage collection as set difference between all transformers and those with outages\n"+'-----'*10)
    df_xfmrs_no_outg = df_xfmrs_all[~df_xfmrs_all['trsf_pole_nb'].isin(df_outage_location_ids['LOCATION_ID'].tolist())]
    assert(df_xfmrs_no_outg['trsf_pole_nb'].shape[0]==df_xfmrs_no_outg['trsf_pole_nb'].nunique())
    print(f"# trsf_pole_nbs in no-outage collection: {df_xfmrs_no_outg['trsf_pole_nb'].nunique()}")
    
    #--------------------------------------------------
    if trsf_pole_nbs_to_ignore is not None:
        print('-----'*20+'\nRemoving unwanted trsf_pole_nbs from no-outage collection\n')
        print(f"BEFORE: {df_xfmrs_no_outg.shape}")
        df_xfmrs_no_outg = df_xfmrs_no_outg[~df_xfmrs_no_outg['trsf_pole_nb'].isin(trsf_pole_nbs_to_ignore)]
        print(f"AFTER : {df_xfmrs_no_outg.shape}")
        print('-----'*20)
        
    #--------------------------------------------------
    # Get outage collection as those in df_xfmrs_all with trsf_pole_nb matching those in df_outage_location_ids
    df_xfmrs_outg    = df_xfmrs_all[df_xfmrs_all['trsf_pole_nb'].isin(df_outage_location_ids['LOCATION_ID'].tolist())]
    #--------------------------------------------------
    print(f'\n\ntime = {time.time()-start}\n'+'-----'*20)
    #--------------------------------------------------
    if save_dfs_to_file:
        df_xfmrs_all.to_pickle(os.path.join(save_dir_base, 'df_xfmrs_all.pkl'))
        df_xfmrs_outg.to_pickle(os.path.join(save_dir_base, 'df_xfmrs_outg.pkl'))
        df_xfmrs_no_outg.to_pickle(os.path.join(save_dir_base, 'df_xfmrs_no_outg.pkl'))
#----------------------------------------------------------------------------------------------------
#--------------------------------------------------
if trsf_pole_nbs_to_ignore is not None:
    print('-----'*20+'\nRemoving unwanted trsf_pole_nbs from no-outage collection\n')
    print(f"BEFORE: {df_xfmrs_no_outg.shape}")
    df_xfmrs_no_outg = df_xfmrs_no_outg[~df_xfmrs_no_outg['trsf_pole_nb'].isin(trsf_pole_nbs_to_ignore)]
    print(f"AFTER : {df_xfmrs_no_outg.shape}")
    print('-----'*20)
    
#--------------------------------------------------
trsf_pole_nbs = df_xfmrs_no_outg['trsf_pole_nb'].unique().tolist()

----------------------------------------------------------------------------------------------------
Finding set of all transformers
--------------------------------------------------


C:\Temp\ipykernel_31748\474070031.py:28: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_xfmrs_all = pd.read_sql(sql_xfmrs_all, conn_aws)


sql_xfmrs_all:
WITH COMP AS (
	SELECT opco_nm,opco_nb
	FROM default.company
)
SELECT
	DISTINCT(trsf_pole_nb),
	COMP.opco_nm
FROM default.meter_premise ereoo
	LEFT JOIN COMP ON ereoo.co_cd_ownr=COMP.opco_nb
WHERE state_cd = 'OH'
AND   serv_city_ad = 'COLUMBUS'
AND   COMP.opco_nm = 'oh'


Determining no outage collection as set difference between all transformers and those with outages
--------------------------------------------------
# trsf_pole_nbs in no-outage collection: 38369
----------------------------------------------------------------------------------------------------
Removing unwanted trsf_pole_nbs from no-outage collection

BEFORE: (38369, 2)
AFTER : (38365, 2)
----------------------------------------------------------------------------------------------------


time = 10.774478435516357
----------------------------------------------------------------------------------------------------
---------------------------------------------------------------------------------------

# Build zip code information for transformers, and possibly restrict data to single zipcode transformers only

### build_or_load_trsf_pole_zips_info

In [27]:
if read_dfs_from_file:
    mp_for_zips_df    = pd.read_pickle(os.path.join(save_dir_base, 'mp_for_zips_df.pkl'))
    trsf_pole_df_full = pd.read_pickle(os.path.join(save_dir_base, 'trsf_location_info_df.pkl'))
    trsf_pole_zips_df = pd.read_pickle(os.path.join(save_dir_base, 'trsf_pole_zips_df.pkl'))
else:
    zips_dict = OutageDAQ.build_trsf_pole_zips_df(
        field_to_split_and_val = ('trsf_pole_nbs', trsf_pole_nbs), 
        states                 = states, 
        opcos                  = opcos, 
        cities                 = cities
    )
    #----------
    trsf_pole_zips_df = zips_dict['trsf_pole_zips_df']
    trsf_pole_df_full = zips_dict['trsf_pole_df_full']
    mp_for_zips_df    = zips_dict['mp_for_zips_df']
    #--------------------------------------------------
    if save_dfs_to_file:
        mp_for_zips_df.to_pickle(   os.path.join(save_dir_base, 'mp_for_zips_df.pkl'))
        trsf_pole_df_full.to_pickle(os.path.join(save_dir_base, 'trsf_location_info_df.pkl'))
        trsf_pole_zips_df.to_pickle(os.path.join(save_dir_base, 'trsf_pole_zips_df.pkl'))
#--------------------------------------------------
if single_zip_xfmrs_only:
    # For our purposes here, we only want entries for which there is a single zip
    trsf_pole_nzips   = trsf_pole_zips_df.drop(columns=['zip+4']).drop_duplicates()['trsf_pole_nb'].value_counts()
    single_zip_poles  = trsf_pole_nzips[trsf_pole_nzips==1].index.tolist()
    #-----
    trsf_pole_zips_df = trsf_pole_zips_df[trsf_pole_zips_df['trsf_pole_nb'].isin(single_zip_poles)]
    #-----
    trsf_pole_nbs = mp_for_zips_df[mp_for_zips_df['trsf_pole_nb'].isin(single_zip_poles)]['trsf_pole_nb'].unique().tolist()

n_coll = 38365
batch_size = 1000
n_batches = 39


C:\Users\s346557\Documents\Analysis\GenAn.py:663: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql, conn_db, **read_sql_args)


10/39
20/39
30/39
trsf_pole_zips_df build time: 14.64705514907837


### build_or_load_df_mp_no_outg

In [ ]:
#----------------------------------------------------------------------------------------------------
# Build joined current and historical MeterPremise collections for all transformers
#   found in df_xfmrs_no_outg.
# Need to use both current and historical because each transformer will be assigned a random
#   datetime (between some date_0 and date_1 limites) below around which to collect events.
#----------------------------------------------------------------------------------------------------
print(
    '-----'*20+'\nBuilding joined current and historical MeterPremise collections '+
    'for all transformers in no-outage collection\n'+'-----'*10
)
start = time.time()
#-----
addtnl_mp_df_cols = ['state_cd', 'circuit_nb', 'circuit_nm', 'station_nb', 'station_nm']
df_mp_no_outg_OG = MeterPremise.build_mp_df_curr_hist_for_xfmrs(
    trsf_pole_nbs          = trsf_pole_nbs, 
    join_curr_hist         = True, 
    drop_approx_duplicates = False, 
    addtnl_mp_df_curr_cols = addtnl_mp_df_cols, 
    addtnl_mp_df_hist_cols = addtnl_mp_df_cols
)
print(f"\nFound MeterPremise data for {df_mp_no_outg_OG['trsf_pole_nb'].nunique()} transformers")
print(f'\ntime = {time.time()-start}\n'+'-----'*20)
#**************************************************
#**************************************************
# Since df_mp_no_outg_OG takes such a significant time to build, save at this point, so all
#   is not lost in case of crash
if save_dfs_to_file:
    df_mp_no_outg_OG.to_pickle(os.path.join(save_dir_base, 'df_mp_no_outg_curr_hist.pkl'))
#**************************************************
#**************************************************

#----------------------------------------------------------------------------------------------------
# For each transformer, set a random date interval between date_0 and date_1 around which
#   results will be acquired.
# This is analogous to the outage event for that dataset.
#----------------------------------------------------------------------------------------------------
print('-----'*20+f'\nSetting random date interval of width={window_width} between date_0={date_0} and date_1={date_1}'+
      '\nfor all remaining transformers in no-outage collection\n')
start = time.time()
df_mp_no_outg_OG = OutageDAQ.set_random_date_interval_in_df_by_xfmr(
    df             = df_mp_no_outg_OG, 
    date_0         = date_0, 
    date_1         = date_1, 
    window_width   = window_width, 
    xfmr_col       = 'trsf_pole_nb', 
    rand_seed      = None, 
    placement_cols = ['start_date', 'end_date'], 
    inplace        = True    
)
# If I'm looking at a single year, with a window_width of 30 days, there are only
# 365-30=335 possible unique start dates, regardless of number of groups.
# So, there are bound to be repeats!
# HOWEVER, I am now using also a random time for each day, so the likeliness of repeats
#   is significantly reduced!
print(f"# trsf_pole_nbs: {df_mp_no_outg_OG['trsf_pole_nb'].nunique()}")
print(f"# start_dates  : {df_mp_no_outg_OG['start_date'].nunique()}")
print(f"# end_dates    : {df_mp_no_outg_OG['end_date'].nunique()}")
print(f'\ntime = {time.time()-start}\n'+'-----'*20)

#----------------------------------------------------------------------------------------------------
# Currently, df_mp_no_outg_OG contains all current and historical MeterPremise data.
# Keep only the data for the meters active during the relevant time periods, as
#   dictated by the start_date and end_date fields.
#----------------------------------------------------------------------------------------------------
print('-----'*20+f'\nKeep only meters active for the relevant times\n')
print(f"BEFORE: df_mp_no_outg_OG.shape = {df_mp_no_outg_OG.shape}")
df_mp_no_outg_OG = df_mp_no_outg_OG[(df_mp_no_outg_OG['inst_ts']<=df_mp_no_outg_OG['start_date']) & 
                                    (df_mp_no_outg_OG['rmvl_ts'].fillna(pd.Timestamp.max)>df_mp_no_outg_OG['end_date'])]
print(f"AFTER : df_mp_no_outg_OG.shape = {df_mp_no_outg_OG.shape}")
print('-----'*20)

#----------------------------------------------------------------------------------------------------
# Drop approximate duplicates from df_mp_no_outg_OG so there is a single entry for each 
#   serial number/premise
#----------------------------------------------------------------------------------------------------
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# PROBLEM:
# drop_approx_mp_duplicates will turn some column elements into lists, which is an issue 
#   when this is plugged into consolidate_df.
# The current solution is to have the addtnl_groupby_cols below, but a more general solution 
#   should be implemented, e.g., if lists, then join the lists or whatever
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
print('-----'*20+f'\nDrop approximate duplicates\n')
print(f"BEFORE: df_mp_no_outg_OG.shape = {df_mp_no_outg_OG.shape}")
df_mp_no_outg_OG = MeterPremise.drop_approx_mp_duplicates(
    mp_df               = df_mp_no_outg_OG, 
    fuzziness           = pd.Timedelta('1 hour'), 
    addtnl_groupby_cols = addtnl_mp_df_cols+['start_date', 'end_date'], 
)
print(f"AFTER : df_mp_no_outg_OG.shape = {df_mp_no_outg_OG.shape}")
print(f'\ntime = {time.time()-start}\n'+'-----'*20)

# Make sure no list entries! (PROBABLY NOT NEEDED ANYMORE AFTER DEV STAGE!)
for col in df_mp_no_outg_OG.columns:
    for i in range(df_mp_no_outg_OG.shape[0]):
        if isinstance(df_mp_no_outg_OG.iloc[i][col], list):
            print(col, i)
            assert(0)

#----------------------------------------------------------------------------------------------------
# Add no_outg_rec_nb column to allow easier grouping when building rcpo_dfs, and for identification later
#----------------------------------------------------------------------------------------------------
print('-----'*20+f'\nCreating no_outg_rec_nb column\n')
start=time.time()
rand_pfx = Utilities.generate_random_string(str_len=5, letters=string.ascii_letters + string.digits)
df_mp_no_outg_OG['no_outg_rec_nb'] = df_mp_no_outg_OG.groupby(['trsf_pole_nb', 'start_date', 'end_date']).ngroup()
df_mp_no_outg_OG['no_outg_rec_nb'] = rand_pfx + df_mp_no_outg_OG['no_outg_rec_nb'].astype(str)
print(f'\ntime = {time.time()-start}\n'+'-----'*20)

#**************************************************
#**************************************************
if save_dfs_to_file:
    df_mp_no_outg_OG.to_pickle(os.path.join(save_dir_base, 'df_mp_no_outg_full.pkl'))
#**************************************************
#**************************************************

# Install and removal timestamps no longer needed, so remove.
df_mp_no_outg_OG = df_mp_no_outg_OG.drop(columns=['inst_ts', 'rmvl_ts'])

# Sort by no_outg_rec_nb, so all like 'no outage events' are sequential in DF
df_mp_no_outg_OG = df_mp_no_outg_OG.sort_values(by=['no_outg_rec_nb', 'trsf_pole_nb', 'prem_nb', 'mfr_devc_ser_nbr'], ignore_index=True)

### build_or_load_pns_with_end_events

In [ ]:
#----------------------------------------------------------------------------------------------------
# Find the premise numbers which recorded an event between date_0 and date_1.
# Premises without any events during the period will be removed from df_mp_no_outg_OG.
# This is relatively quick, and saves much time in the data acquisition step as we do not
#   have to waste time searching for empty results (which is suprisingly sluggish)
#----------------------------------------------------------------------------------------------------
print('-----'*20+f'\nFind PNs with recorded events between date_0={date_0} and date_1={date_1}\n'+'-----'*10)
start=time.time()
pns_with_end_events, sql_pns_with_end_events = AMIEndEvents.get_end_event_distinct_fields(
    date_0=date_0, 
    date_1=date_1, 
    fields=['aep_premise_nb'], 
    are_datetime=False, 
    addtnl_build_sql_function_kwargs = dict(
        states = states, 
        opcos  = opcos, 
        cities = cities
    ), 
    return_sql=True
)
#----------
print('Removing PNs without recorded events to speed up data acquisition\n'+'-----'*10)
print(f'BEFORE: df_mp_no_outg_OG.shape = {df_mp_no_outg_OG.shape}')
print(f"        df_mp_no_outg_OG['trsf_pole_nb'].nunique() = {df_mp_no_outg_OG['trsf_pole_nb'].nunique()}")
#----------
df_mp_no_outg_OG = df_mp_no_outg_OG[df_mp_no_outg_OG['prem_nb'].isin(pns_with_end_events['aep_premise_nb'].tolist())]
#----------
print(f'AFTER:  df_mp_no_outg_OG.shape = {df_mp_no_outg_OG.shape}')
print(f"        df_mp_no_outg_OG['trsf_pole_nb'].nunique() = {df_mp_no_outg_OG['trsf_pole_nb'].nunique()}")
#----------
print(f'\n\ntime = {time.time()-start}\n'+'-----'*20)

### build_or_load_df_mp_no_outg_w_events_slim

In [ ]:
#----------------------------------------------------------------------------------------------------
# Build consolidated (slim) version of df_mp_no_outage
#----------------------------------------------------------------------------------------------------
cols_shared_by_group = addtnl_mp_df_cols
cols_to_collect_in_lists = ['mfr_devc_ser_nbr', 'prem_nb']
rename_cols = {'mfr_devc_ser_nbr':'serial_numbers', 'prem_nb':'premise_nbs'}

print('-----'*20+f'\nBuilding consolidated version of df_mp_no_outg\n'+'-----'*10)
start=time.time()
df_mp_no_outg_slim_OG = Utilities_df.consolidate_df(
    df                                  = df_mp_no_outg_OG, 
    groupby_cols                        = ['no_outg_rec_nb', 'trsf_pole_nb', 'start_date', 'end_date'], 
    cols_shared_by_group                = cols_shared_by_group, 
    cols_to_collect_in_lists            = cols_to_collect_in_lists, 
    include_groupby_cols_in_output_cols = False, 
    allow_duplicates_in_lists           = False, 
    recover_uniqueness_violators        = True, 
    rename_cols                         = rename_cols, 
    verbose                             = True
)
#-------------------------
df_mp_no_outg_slim_OG=df_mp_no_outg_slim_OG.reset_index().set_index(['no_outg_rec_nb', 'trsf_pole_nb'], drop=False)
df_mp_no_outg_slim_OG.index.names=['idx_no_outg_rec_nb', 'idx_trsf_pole_nb']
#-----
df_mp_no_outg_slim_OG['premise_nbs']    = df_mp_no_outg_slim_OG['premise_nbs'].apply(sorted)
df_mp_no_outg_slim_OG['serial_numbers'] = df_mp_no_outg_slim_OG['serial_numbers'].apply(sorted)
#-------------------------
print(f'\ntime = {time.time()-start}\n'+'-----'*20)

In [ ]:
#****************************************************************************************************
# Save DFs
#****************************************************************************************************
if save_dfs_to_file:
    # df_outage_location_ids.to_pickle(os.path.join(save_dir_base, 'df_outage_location_ids.pkl'))
    # df_xfmrs_all.to_pickle(os.path.join(save_dir_base, 'df_xfmrs_all.pkl'))
    # df_xfmrs_outg.to_pickle(os.path.join(save_dir_base, 'df_xfmrs_outg.pkl'))
    # df_xfmrs_no_outg.to_pickle(os.path.join(save_dir_base, 'df_xfmrs_no_outg.pkl'))
    pns_with_end_events.to_pickle(os.path.join(save_dir_base, 'pns_with_end_events.pkl'))
    #-----
    df_mp_no_outg_OG.to_pickle(os.path.join(save_dir_base, 'df_mp_no_outg_PNs_w_events.pkl'))
    df_mp_no_outg_slim_OG.to_pickle(os.path.join(save_dir_base, 'df_mp_no_outg_PNs_w_events_slim.pkl'))
    
    #-------------------------
    # Below, date_0 and date_1 should be string, but use str()
    #   in case they were converted to pd.Datetime or whatever
    run_info = dict(
        date_0                  = str(date_0), 
        date_1                  = str(date_1), 
        window_width            = str(window_width), 
        run_using_slim          = run_using_slim,
        states                  = states, 
        opcos                   = opcos, 
        cities                  = cities, 
        trsf_pole_nbs_to_ignore = trsf_pole_nbs_to_ignore
    )
    CustomWriter.output_dict_to_json(os.path.join(save_dir_base, 'run_info.json'), run_info)

In [ ]:
# df_mp_no_outg_OG = pd.read_pickle(os.path.join(save_dir_base, 'df_mp_no_outg_PNs_w_events.pkl'))
# df_mp_no_outg_slim_OG = pd.read_pickle(os.path.join(save_dir_base, 'df_mp_no_outg_PNs_w_events_slim.pkl'))


# -----------------------------------------------------------------------------------------------
# Collect events
# -----------------------------------------------------------------------------------------------

### collect_events

In [ ]:
df_mp_no_outg_slim = df_mp_no_outg_slim_OG.copy()
df_mp_no_outg      = df_mp_no_outg_OG.copy()
#----------
df_construct_type              = DFConstructType.kRunSqlQuery
contstruct_df_args_end_events  = None
addtnl_groupby_cols            = ['trsf_pole_nb', 'no_outg_rec_nb']

cols_of_interest_end_dev_event = TableInfos.AMIEndEvents_TI.std_columns_of_interest
verbose                        = True
n_update                       = 1

In [ ]:
#--------------------------------------------------
end_events_sql_function_kwargs = dict(
    cols_of_interest                  = cols_of_interest_end_dev_event, 
    join_mp_args                      = False, 
    df_args                           = dict(
        addtnl_groupby_cols = addtnl_groupby_cols, 
        t_search_min_col    = 'start_date', 
        t_search_max_col    = 'end_date'
    ), 
    field_to_split                    = 'df_mp_no_outg', 
    field_to_split_location_in_kwargs = ['df_mp_no_outg'], 
    sort_coll_to_split                = False,
    verbose                           = verbose, 
    n_update                          = n_update
)
#----------
# if opcos is not None:
addtnl_end_events_sql_function_kwargs = dict(
    build_sql_function_kwargs = dict(
        states = states, 
        opcos  = opcos, 
        cities = cities
    )
)
end_events_sql_function_kwargs = {
    **end_events_sql_function_kwargs, 
    **addtnl_end_events_sql_function_kwargs
}

In [ ]:
#--------------------------------------------------
if run_using_slim:
    batch_size = 10
    #----------
    end_events_sql_function_kwargs = Utilities.supplement_dict_with_default_values(
        to_supplmnt_dict    = end_events_sql_function_kwargs, 
        default_values_dict = dict(
            df_mp_no_outg   = df_mp_no_outg_slim, 
            batch_size      = batch_size, 
            df_args         = dict(
                mapping_to_ami     = DfToSqlMap(df_col='premise_nbs', kwarg='premise_nbs', sql_col='aep_premise_nb'), 
                is_df_consolidated = True
            )
        ), 
        extend_any_lists    = True,
        inplace             = True
    )
#-------------------------
else:
    batch_size=200
    #----------
    end_events_sql_function_kwargs = Utilities.supplement_dict_with_default_values(
        to_supplmnt_dict = end_events_sql_function_kwargs, 
        default_values_dict = dict(
            df_mp_no_outg   = df_mp_no_outg, 
            batch_size      = batch_size, 
            df_args         = dict(
                mapping_to_ami     = DfToSqlMap(df_col='prem_nb', kwarg='premise_nbs', sql_col='aep_premise_nb'), 
                is_df_consolidated = False
            )
        ), 
        extend_any_lists    = True,
        inplace             = True
    )

In [ ]:
start=time.time()

exit_status = Utilities.run_tryexceptwhile_process(
    func                = AMIEndEvents,
    func_args_dict      = dict(
        df_construct_type         = df_construct_type, 
        contstruct_df_args        = contstruct_df_args_end_events, 
        build_sql_function        = AMIEndEvents_SQL.build_sql_end_events_for_no_outages, 
        build_sql_function_kwargs = end_events_sql_function_kwargs, 
        init_df_in_constructor    = True, 
        save_args                 = end_events_save_args
    ), 
    max_calls_per_min   = 1, 
    lookback_period_min = 15, 
    max_calls_absolute  = 1000, 
    verbose             = True
)
print(f'exit_status = {exit_status}')

build_time = time.time()-start
print(build_time)

In [ ]:
assert(0)